In [2]:
#Access local data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
df=pd.read_csv("Metro_Interstate_Traffic_Volume.csv")  
df.head(10)
df.columns

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'date_time', 'traffic_volume'],
      dtype='object')

In [5]:
df

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,delta
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02 09:00:00,0
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02 10:00:00,0
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02 11:00:00,0
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02 12:00:00,0
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02 13:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...
48199,None,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,3543,2018-09-30 19:00:00,2189
48200,None,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2781,2018-09-30 20:00:00,2189
48201,None,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159,2018-09-30 21:00:00,2189
48202,None,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450,2018-09-30 22:00:00,2189


In [4]:
from datetime import datetime

datetime.strptime(df['date_time'].iloc[1], '%Y-%m-%d %H:%M:%S')
df['date'] = df['date_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['delta'] = (df['date'] - df['date'].min()).dt.days
#df['delta']

0           0
1           0
2           0
3           0
4           0
         ... 
48199    2189
48200    2189
48201    2189
48202    2189
48203    2189
Name: delta, Length: 48204, dtype: int64

In [20]:
from sklearn.preprocessing import OneHotEncoder

Encode=OneHotEncoder(handle_unknown='ignore')
X_Encode=Encode.fit_transform(df[['holiday', 'weather_main','weather_description']]).toarray()
X_Encode.shape

X = np.concatenate((X_Encode,df[['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'delta',]]),axis=1)
X.shape

(48204, 66)

In [11]:
# standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[0]

array([-0.01115735, -0.01018512, -0.01018512, -0.01205144, -0.01115735,
       -0.01018512, -0.01115735,  0.03559577, -0.01018512, -0.01115735,
       -0.01018512, -0.01018512, -0.62020574,  1.476091  , -0.19814155,
       -0.13886845, -0.17038936, -0.37525334, -0.36518271, -0.0203734 ,
       -0.25189014, -0.00910975, -0.14805645, -0.00910975, -0.19270664,
       -0.32736944, -0.11700422, -0.20565437, -0.13886845, -0.00644143,
       -0.17038936, -0.0364617 , -0.09890788, -0.11377363, -0.15281551,
       -0.01642437, -0.27425195, -0.01115735, -0.01510791, -0.20510582,
       -0.37525334, -0.18908778, -0.34325756, -0.0531914 , -0.11899235,
       -0.01642437, -0.03286204,  3.59552034, -0.01115735, -0.00455473,
       -0.56502029, -0.00788919, -0.0203734 , -0.0782017 , -0.05098909,
       -0.00644143, -0.03617535, -0.01764297, -0.03348873, -0.02771571,
       -0.01932751,  0.53037038, -0.00746314, -0.02722834, -0.23996279,
       -1.74204093])

In [12]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['traffic_volume'], test_size=0.2, random_state=42)

In [21]:
#Multiple regression model
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [25]:
Y_pred_lr=lr.predict(X_test) # this function is used to predict the price parameter x is array

print("For Multiple Linear Regression:")

# MSE for test data
from sklearn.metrics import mean_squared_error
print("MSE for test data is",mean_squared_error(y_test,Y_pred_lr))

# R-squared for test data
from sklearn.metrics import r2_score
print("R-squared for test data is",r2_score(y_test,Y_pred_lr))

For Multiple Linear Regression:
MSE for test data is 3747243.216672318
R-squared for test data is 0.05217393068890741


Polynomial Regression and Pipelines


In [23]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

#We create the pipeline, by creating a list of tuples including the name of the model or estimator and its corresponding constructor.
Input=[('polynomial', PolynomialFeatures(2)), ('model',LinearRegression())]
#we input the list as an argument to the pipeline constructo
pipe=Pipeline(Input)
pipe


Pipeline(steps=[('polynomial', PolynomialFeatures()),
                ('model', LinearRegression())])

In [24]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('polynomial', PolynomialFeatures()),
                ('model', LinearRegression())])

In [19]:
y_pred_poly = pipe.predict(X_test)

print("For Polynomial Regression:")
# MSE for test data
from sklearn.metrics import mean_squared_error
print("MSE for test data is",mean_squared_error(y_test,y_pred_poly))

# R-squared for test data
from sklearn.metrics import r2_score
print("R-squared for test data is",r2_score(y_test,y_pred_poly))

For Polynomial Regression:
MSE for test data is 2.531437925812273e+26
R-squared for test data is -6.403008078718704e+19
